In [1]:
import numpy as np
import pandas as pd
import re

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data = pd.read_csv('/kaggle/input/netflix-shows/netflix_titles.csv')
data.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [3]:
data.groupby('type').count()

,show_id,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
type,,,,,,,,,,,
Movie,6131,6131,5943,5656,5691,6131,6131,6129,6128,6131,6131
TV Show,2676,2676,230,2326,2285,2666,2676,2674,2676,2676,2676


In [4]:
data = data.dropna(subset=['cast', 'country', 'rating'])

In [5]:
movies = data[data['type'] == 'Movie'].reset_index()
movies = movies.drop(['index', 'show_id', 'type', 'date_added', 'release_year', 'duration', 'description'], axis=1)
movies.head()

,title,director,cast,country,rating,listed_in
0,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...",TV-MA,"Dramas, Independent Movies, International Movies"
1,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...",United States,PG-13,"Comedies, Dramas"
2,Je Suis Karl,Christian Schwochow,"Luna Wedler, Jannis Niewöhner, Milan Peschel, ...","Germany, Czech Republic",TV-MA,"Dramas, International Movies"
3,Jeans,S. Shankar,"Prashanth, Aishwarya Rai Bachchan, Sri Lakshmi...",India,TV-14,"Comedies, International Movies, Romantic Movies"
4,Grown Ups,Dennis Dugan,"Adam Sandler, Kevin James, Chris Rock, David S...",United States,PG-13,Comedies


In [6]:
tv = data[data['type'] == 'TV Show'].reset_index()
tv = tv.drop(['index', 'show_id', 'type', 'date_added', 'release_year', 'duration', 'description'], axis=1)
tv.head()

,title,director,cast,country,rating,listed_in
0,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,TV-MA,"International TV Shows, TV Dramas, TV Mysteries"
1,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,TV-MA,"International TV Shows, Romantic TV Shows, TV ..."
2,The Great British Baking Show,Andy Devonshire,"Mel Giedroyc, Sue Perkins, Mary Berry, Paul Ho...",United Kingdom,TV-14,"British TV Shows, Reality TV"
3,Dear White People,NaN,"Logan Browning, Brandon P. Bell, DeRon Horton,...",United States,TV-MA,"TV Comedies, TV Dramas"
4,Falsa identidad,NaN,"Luis Ernesto Franco, Camila Sodi, Sergio Goyri...",Mexico,TV-MA,"Crime TV Shows, Spanish-Language TV Shows, TV ..."


In [7]:
actors = []

for i in movies['cast']:
    actor = re.split(r', \s*', i)
    actors.append(actor)
    
flat_list = []
for sublist in actors:
    for item in sublist:
        flat_list.append(item)
        
actors_list = sorted(set(flat_list))

binary_actors = [[0] * 0 for i in range(len(set(flat_list)))]

for i in movies['cast']:
    k = 0
    for j in actors_list:
        if j in i:
            binary_actors[k].append(1.0)
        else:
            binary_actors[k].append(0.0)
        k+=1
        
binary_actors = pd.DataFrame(binary_actors).transpose()
        
directors = []

for i in movies['director']:
    if pd.notna(i):
        director = re.split(r', \s*', i)
        directors.append(director)
    
flat_list2 = []
for sublist in directors:
    for item in sublist:
        flat_list2.append(item)
        
directors_list = sorted(set(flat_list2))

binary_directors = [[0] * 0 for i in range(len(set(flat_list2)))]

for i in movies['director']:
    k = 0
    for j in directors_list:
        if pd.isna(i):
            binary_directors[k].append(0.0)
        elif j in i:
            binary_directors[k].append(1.0)
        else:
            binary_directors[k].append(0.0)
        k+=1
        
binary_directors = pd.DataFrame(binary_directors).transpose()
        
countries = []

for i in movies['country']:
    country = re.split(r', \s*', i)
    countries.append(country)
    
flat_list3 = []
for sublist in countries:
    for item in sublist:
        flat_list3.append(item)
        
countries_list = sorted(set(flat_list3))

binary_countries = [[0] * 0 for i in range(len(set(flat_list3)))]

for i in movies['country']:
    k = 0
    for j in countries_list:
        if j in i:
            binary_countries[k].append(1.0)
        else:
            binary_countries[k].append(0.0)
        k+=1
        
binary_countries = pd.DataFrame(binary_countries).transpose()

genres = []

for i in movies['listed_in']:
    genre = re.split(r', \s*', i)
    genres.append(genre)
    
flat_list4 = []
for sublist in genres:
    for item in sublist:
        flat_list4.append(item)
        
genres_list = sorted(set(flat_list4))

binary_genres = [[0] * 0 for i in range(len(set(flat_list4)))]

for i in movies['listed_in']:
    k = 0
    for j in genres_list:
        if j in i:
            binary_genres[k].append(1.0)
        else:
            binary_genres[k].append(0.0)
        k+=1
        
binary_genres = pd.DataFrame(binary_genres).transpose()

ratings = []

for i in movies['rating']:
    ratings.append(i)

ratings_list = sorted(set(ratings))

binary_ratings = [[0] * 0 for i in range(len(set(ratings_list)))]

for i in movies['rating']:
    k = 0
    for j in ratings_list:
        if j in i:
            binary_ratings[k].append(1.0)
        else:
            binary_ratings[k].append(0.0)
        k+=1
        
binary_ratings = pd.DataFrame(binary_ratings).transpose()

In [8]:
binary = pd.concat([binary_actors, binary_directors, binary_countries, binary_genres], axis=1,ignore_index=True)
binary

,0,1,2,3,4,5,6,7,8,9,...,29105,29106,29107,29108,29109,29110,29111,29112,29113,29114
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5275,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
5276,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
5277,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5278,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
actors2 = []

for i in tv['cast']:
    actor2 = re.split(r', \s*', i)
    actors2.append(actor2)
    
flat_list5 = []
for sublist in actors2:
    for item in sublist:
        flat_list5.append(item)
        
actors_list2 = sorted(set(flat_list5))

binary_actors2 = [[0] * 0 for i in range(len(set(flat_list5)))]

for i in tv['cast']:
    k = 0
    for j in actors_list2:
        if j in i:
            binary_actors2[k].append(1.0)
        else:
            binary_actors2[k].append(0.0)
        k+=1
        
binary_actors2 = pd.DataFrame(binary_actors2).transpose()
        

countries2 = []

for i in tv['country']:
    country2 = re.split(r', \s*', i)
    countries2.append(country2)
    
flat_list6 = []
for sublist in countries2:
    for item in sublist:
        flat_list6.append(item)
        
countries_list2 = sorted(set(flat_list6))

binary_countries2 = [[0] * 0 for i in range(len(set(flat_list6)))]

for i in tv['country']:
    k = 0
    for j in countries_list2:
        if j in i:
            binary_countries2[k].append(1.0)
        else:
            binary_countries2[k].append(0.0)
        k+=1
        
binary_countries2 = pd.DataFrame(binary_countries2).transpose()

genres2 = []

for i in tv['listed_in']:
    genre2 = re.split(r', \s*', i)
    genres2.append(genre2)
    
flat_list7 = []
for sublist in genres2:
    for item in sublist:
        flat_list7.append(item)
        
genres_list2 = sorted(set(flat_list7))

binary_genres2 = [[0] * 0 for i in range(len(set(flat_list7)))]

for i in tv['listed_in']:
    k = 0
    for j in genres_list2:
        if j in i:
            binary_genres2[k].append(1.0)
        else:
            binary_genres2[k].append(0.0)
        k+=1
        
binary_genres2 = pd.DataFrame(binary_genres2).transpose()

ratings2 = []

for i in tv['rating']:
    ratings2.append(i)

ratings_list2 = sorted(set(ratings2))

binary_ratings2 = [[0] * 0 for i in range(len(set(ratings_list2)))]

for i in tv['rating']:
    k = 0
    for j in ratings_list2:
        if j in i:
            binary_ratings2[k].append(1.0)
        else:
            binary_ratings2[k].append(0.0)
        k+=1
        
binary_ratings2 = pd.DataFrame(binary_ratings2).transpose()

In [10]:
binary2 = pd.concat([binary_actors2, binary_countries2, binary_genres2], axis=1, ignore_index=True)
binary2

,0,1,2,3,4,5,6,7,8,9,...,13561,13562,13563,13564,13565,13566,13567,13568,13569,13570
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
def recommender(search):
    cs_list = []
    binary_list = []
    if search in movies['title'].values:
        idx = movies[movies['title'] == search].index.item()
        for i in binary.iloc[idx]:
            binary_list.append(i)
        point1 = np.array(binary_list).reshape(1, -1)
        point1 = [val for sublist in point1 for val in sublist]    
        for j in range(len(movies)):
            binary_list2 = []
            for k in binary.iloc[j]:
                binary_list2.append(k)
            point2 = np.array(binary_list2).reshape(1, -1)
            point2 = [val for sublist in point2 for val in sublist]
            dot_product = np.dot(point1, point2)
            norm_1 = np.linalg.norm(point1)
            norm_2 = np.linalg.norm(point2)
            cos_sim = dot_product / (norm_1 * norm_2)
            cs_list.append(cos_sim)
        movies_copy = movies.copy()
        movies_copy['cos_sim'] = cs_list
        results = movies_copy.sort_values('cos_sim', ascending=False)
        results = results[results['title'] != search]    
        top_results = results.head(5)
        return(top_results)
    elif search in tv['title'].values:
        idx = tv[tv['title'] == search].index.item()
        for i in binary2.iloc[idx]:
            binary_list.append(i)
        point1 = np.array(binary_list).reshape(1, -1)
        point1 = [val for sublist in point1 for val in sublist]
        for j in range(len(tv)):
            binary_list2 = []
            for k in binary2.iloc[j]:
                binary_list2.append(k)
            point2 = np.array(binary_list2).reshape(1, -1)
            point2 = [val for sublist in point2 for val in sublist]
            dot_product = np.dot(point1, point2)
            norm_1 = np.linalg.norm(point1)
            norm_2 = np.linalg.norm(point2)
            cos_sim = dot_product / (norm_1 * norm_2)
            cs_list.append(cos_sim)
        tv_copy = tv.copy()
        tv_copy['cos_sim'] = cs_list
        results = tv_copy.sort_values('cos_sim', ascending=False)
        results = results[results['title'] != search]    
        top_results = results.head(5)
        return(top_results)
    else:
        return("Title not in dataset. Please check spelling.")

In [12]:
recommender('The Conjuring')

,title,director,cast,country,rating,listed_in,cos_sim
507,Insidious,James Wan,"Patrick Wilson, Rose Byrne, Lin Shaye, Ty Simp...","United States, Canada",PG-13,"Horror Movies, Thrillers",0.424437
3223,Creep,Patrick Brice,"Mark Duplass, Patrick Brice",United States,R,"Horror Movies, Independent Movies, Thrillers",0.416667
603,The Conjuring 2,James Wan,"Patrick Wilson, Vera Farmiga, Madison Wolfe, F...","Canada, United States, United Kingdom",R,Horror Movies,0.412479
1832,In the Tall Grass,Vincenzo Natali,"Patrick Wilson, Laysla De Oliveira, Avery Whit...","Canada, United States",TV-MA,"Horror Movies, Thrillers",0.400892
3728,Desolation,Sam Patton,"Jaimi Paige, Alyshia Ochse, Toby Nichols, Clau...",United States,TV-MA,"Horror Movies, Thrillers",0.395285


In [15]:
recommender("Child's Play")

,title,director,cast,country,rating,listed_in,cos_sim
3689,Cult of Chucky,Don Mancini,"Fiona Dourif, Michael Therriault, Adam Hurtig,...",United States,R,Horror Movies,0.352941
1461,Wildling,Fritz Böhm,"Bel Powley, Brad Dourif, Liv Tyler, Collin Kel...",United States,R,"Horror Movies, Independent Movies, Sci-Fi & Fa...",0.350070
4768,Stephanie,Akiva Goldsman,"Shree Cooks, Frank Grillo, Anna Torv",United States,R,Horror Movies,0.342997
4434,Nova: Ultimate Mars Challenge,Gail Willumsen,Lance Lewman,United States,TV-G,Movies,0.325396
3160,Louis C.K.: Hilarious,Louis C.K.,Louis C.K.,United States,84 min,Movies,0.325396


In [16]:
recommender('Charlie and the Chocolate Factory')

,title,director,cast,country,rating,listed_in,cos_sim
3835,Figaro Pho,Luke Jurevicius,Luke Jurevicius,United Kingdom,TV-Y7,"Children & Family Movies, Comedies",0.394055
4506,Penelope,Mark Palansky,"Christina Ricci, James McAvoy, Catherine O'Har...","United Kingdom, United States",PG,"Children & Family Movies, Comedies, Romantic M...",0.390095
4802,Sweeney Todd: The Demon Barber of Fleet Street,Tim Burton,"Johnny Depp, Helena Bonham Carter, Alan Rickma...","United States, United Kingdom",R,"Dramas, Horror Movies, Music & Musicals",0.382692
2606,The Kissing Booth,Vince Marcello,"Joey King, Joel Courtney, Jacob Elordi, Molly ...","United Kingdom, United States",TV-14,"Comedies, Romantic Movies",0.361158
1547,A Shaun the Sheep Movie: Farmageddon,"Richard Phelan, Will Becher","Justin Fletcher, John Sparkes, Amalia Vitale, ...","United Kingdom, France, Belgium, Ireland, Unit...",G,"Children & Family Movies, Comedies",0.354005


In [17]:
recommender('Wild Child')

,title,director,cast,country,rating,listed_in,cos_sim
2606,The Kissing Booth,Vince Marcello,"Joey King, Joel Courtney, Jacob Elordi, Molly ...","United Kingdom, United States",TV-14,"Comedies, Romantic Movies",0.440959
4506,Penelope,Mark Palansky,"Christina Ricci, James McAvoy, Catherine O'Har...","United Kingdom, United States",PG,"Children & Family Movies, Comedies, Romantic M...",0.408248
1130,The Kissing Booth 2,Vince Marcello,"Joey King, Joel Courtney, Jacob Elordi, Molly ...","United Kingdom, United States",TV-14,"Comedies, Romantic Movies",0.394405
122,The Kissing Booth 3,Vince Marcello,"Joey King, Joel Courtney, Jacob Elordi, Molly ...","United Kingdom, United States",TV-14,"Comedies, Romantic Movies",0.381881
2501,The Guernsey Literary and Potato Peel Pie Society,Mike Newell,"Lily James, Michiel Huisman, Penelope Wilton, ...","United Kingdom, France, United States",TV-PG,"Dramas, Romantic Movies",0.381881


In [18]:
recommender("Dr. Seuss' The Cat in the Hat")

,title,director,cast,country,rating,listed_in,cos_sim
4419,NOVA: Bird Brain,NaN,Craig Sechler,United States,TV-G,"Children & Family Movies, Documentaries",0.436436
1114,Sugar High,Ariel Boles,Hunter March,United States,TV-G,Children & Family Movies,0.436436
4758,Spy Kids 3: Game Over,Robert Rodriguez,"Daryl Sabara, Sylvester Stallone, Ricardo Mont...",United States,PG,"Children & Family Movies, Comedies",0.422577
2245,Prince of Peoria: A Christmas Moose Miracle,Jon Rosenbaum,"Gavin Lewis, Theodore Barnes, Shelby Simmons, ...",United States,TV-G,"Children & Family Movies, Comedies",0.422577
1363,What a Girl Wants,Dennie Gordon,"Amanda Bynes, Colin Firth, Kelly Preston, Eile...","United States, United Kingdom",PG,"Children & Family Movies, Comedies",0.414039


In [19]:
recommender('Hook')

,title,director,cast,country,rating,listed_in,cos_sim
1918,Sotus The Series,NaN,"Sangpotirat Perawat, Ruangroj Prachaya",Thailand,TV-MA,"International TV Shows, Romantic TV Shows, TV ...",0.405554
843,Little Things,NaN,"Dhruv Sehgal, Mithila Palkar",India,TV-MA,"International TV Shows, Romantic TV Shows, TV ...",0.405554
1898,Roonpi Secret Love,NaN,"Kanyawee Songmuang, Korapat Kirdpan",Thailand,TV-14,"International TV Shows, Romantic TV Shows, Tee...",0.405554
200,Let's Fight Ghost,Kongkiat Khomsiri,"Suppapong Udomkaewkanjana, Patchanan Jiajirach...",Thailand,TV-14,"International TV Shows, Romantic TV Shows, TV ...",0.381771
1477,Highland: Thailand's Marijuana Awakening,NaN,Sebastian Perry,Thailand,TV-MA,"Docuseries, International TV Shows",0.374634


In [18]:
recommender('Holidate')

,title,director,cast,country,rating,listed_in,cos_sim
2722,When We First Met,Ari Sandel,"Adam DeVine, Alexandra Daddario, Shelley Henni...",United States,TV-14,"Comedies, Romantic Movies",0.467099
3844,First Impression,Arthur Muhammad,"Lamman Rucker, Lisa Arrindell Anderson, Elise ...",United States,TV-14,"Comedies, Romantic Movies",0.408248
2855,Chadi Jawani Budhe Nu,NaN,Anand,India,TV-PG,"Comedies, International Movies, Romantic Movies",0.390360
1783,Holly Star,Michael A. Nickles,"Katlyn Carlson, Brian Muller, Teya Patt, Pamel...",United States,PG,"Children & Family Movies, Comedies, Romantic M...",0.389249
4938,The Good Catholic,Paul Shoulberg,"Zachary Spicer, Wrenn Schmidt, Danny Glover, J...",United States,PG-13,"Comedies, Dramas, Romantic Movies",0.389249


In [19]:
recommender('After Life')

,title,director,cast,country,rating,listed_in,cos_sim
1699,Extras,NaN,"Ricky Gervais, Stephen Merchant, Ashley Jensen...","United Kingdom, United States",TV-MA,"British TV Shows, TV Comedies",0.564076
1816,Mary Portas: Secret Shopper,NaN,Mary Portas,United Kingdom,TV-14,"British TV Shows, International TV Shows, Real...",0.505076
1636,"Big Dreams, Small Spaces",NaN,Monty Don,United Kingdom,TV-G,"British TV Shows, International TV Shows, Real...",0.505076
1717,Genius of the Ancient World,NaN,Bettany Hughes,United Kingdom,TV-PG,"British TV Shows, Docuseries, International TV...",0.505076
1718,Genius of the Modern World,NaN,Bettany Hughes,United Kingdom,TV-PG,"British TV Shows, Docuseries, International TV...",0.505076


In [20]:
recommender('Anne with an E')

,title,director,cast,country,rating,listed_in,cos_sim
400,Can You Hear Me?,NaN,"Mélissa Bédard, Ève Landry, Florence Longpré",Canada,TV-MA,"International TV Shows, TV Comedies, TV Dramas",0.507093
621,Restaurants on the Edge,NaN,"Nick Liberato, Karin Bohn, Dennis Prescott",Canada,TV-14,"International TV Shows, Reality TV",0.445435
1739,Heavy Rescue: 401,NaN,Dave Pettitt,Canada,TV-MA,"International TV Shows, Reality TV",0.436436
750,Hip-Hop Evolution,NaN,Shad Kabango,Canada,TV-MA,"Docuseries, International TV Shows",0.436436
1637,Bitten,NaN,"Laura Vandervoort, Greyston Holt, Greg Bryk, S...",Canada,TV-MA,"International TV Shows, TV Dramas, TV Horror",0.428571


In [16]:
movies_des = data[data['type'] == 'Movie'].reset_index()
movies_des = movies_des[['title', 'description']]
movies_des.head()

,title,description
0,Sankofa,"On a photo shoot in Ghana, an American model s..."
1,The Starling,A woman adjusting to life after a loss contend...
2,Je Suis Karl,After most of her family is murdered in a terr...
3,Jeans,When the father of the man she loves insists t...
4,Grown Ups,Mourning the loss of their beloved junior high...


In [17]:
tv_des = data[data['type'] == 'TV Show'].reset_index()
tv_des = tv_des[['title', 'description']]
tv_des.head()

,title,description
0,Blood & Water,"After crossing paths at a party, a Cape Town t..."
1,Kota Factory,In a city of coaching centers known to train I...
2,The Great British Baking Show,A talented batch of amateur bakers face off in...
3,Dear White People,Students of color navigate the daily slights a...
4,Falsa identidad,Strangers Diego and Isabel flee their home in ...


In [18]:
filtered_movies = []
movies_words = []

for text in movies_des['description']:
    text_tokens = word_tokenize(text)
    tokens_without_sw = [word.lower() for word in text_tokens if not word in stopwords.words()]
    movies_words.append(tokens_without_sw)
    filtered = (" ").join(tokens_without_sw)
    filtered_movies.append(filtered)

movies_words = [val for sublist in movies_words for val in sublist]
movies_words = sorted(set(movies_words))
movies_des['description_filtered'] = filtered_movies
movies_des.head()

,title,description,description_filtered
0,Sankofa,"On a photo shoot in Ghana, an American model s...","on photo shoot ghana , american model slips ba..."
1,The Starling,A woman adjusting to life after a loss contend...,a woman adjusting life loss contends feisty bi...
2,Je Suis Karl,After most of her family is murdered in a terr...,"after family murdered terrorist bombing , youn..."
3,Jeans,When the father of the man she loves insists t...,when father loves insists twin marry twin sist...
4,Grown Ups,Mourning the loss of their beloved junior high...,mourning loss beloved junior high basketball c...


In [19]:
filtered_tv = []
tv_words = []

for text in tv_des['description']:
    text_tokens = word_tokenize(text)
    tokens_without_sw = [word.lower() for word in text_tokens if not word in stopwords.words()]
    tv_words.append(tokens_without_sw)
    filtered = (" ").join(tokens_without_sw)
    filtered_tv.append(filtered)

tv_words = [val for sublist in tv_words for val in sublist]
tv_words = sorted(set(tv_words))
tv_des['description_filtered'] = filtered_tv
tv_des.head()

,title,description,description_filtered
0,Blood & Water,"After crossing paths at a party, a Cape Town t...","after crossing paths party , cape town sets pr..."
1,Kota Factory,In a city of coaching centers known to train I...,in city coaching centers train india ’ finest ...
2,The Great British Baking Show,A talented batch of amateur bakers face off in...,a talented batch amateur bakers 10-week compet...
3,Dear White People,Students of color navigate the daily slights a...,students color navigate daily slights slippery...
4,Falsa identidad,Strangers Diego and Isabel flee their home in ...,strangers diego isabel flee home mexico preten...


In [20]:
movie_word_binary = [[0] * 0 for i in range(len(set(movies_words)))]

for des in movies_des['description_filtered']:
    k = 0
    for word in movies_words:
        if word in des:
            movie_word_binary[k].append(1.0)
        else:
            movie_word_binary[k].append(0.0)
        k+=1
        
movie_word_binary = pd.DataFrame(movie_word_binary).transpose()

In [21]:
tv_word_binary = [[0] * 0 for i in range(len(set(tv_words)))]

for des in tv_des['description_filtered']:
    k = 0
    for word in tv_words:
        if word in des:
            tv_word_binary[k].append(1.0)
        else:
            tv_word_binary[k].append(0.0)
        k+=1
        
tv_word_binary = pd.DataFrame(tv_word_binary).transpose()

In [22]:
def recommender2(search):
    cs_list = []
    binary_list = []
    if search in movies_des['title'].values:
        idx = movies_des[movies_des['title'] == search].index.item()
        for i in movie_word_binary.iloc[idx]:
            binary_list.append(i)
        point1 = np.array(binary_list).reshape(1, -1)
        point1 = [val for sublist in point1 for val in sublist]    
        for j in range(len(movies_des)):
            binary_list2 = []
            for k in movie_word_binary.iloc[j]:
                binary_list2.append(k)
            point2 = np.array(binary_list2).reshape(1, -1)
            point2 = [val for sublist in point2 for val in sublist]
            dot_product = np.dot(point1, point2)
            norm_1 = np.linalg.norm(point1)
            norm_2 = np.linalg.norm(point2)
            cos_sim = dot_product / (norm_1 * norm_2)
            cs_list.append(cos_sim)
        movies_copy = movies_des.copy()
        movies_copy['cos_sim'] = cs_list
        results = movies_copy.sort_values('cos_sim', ascending=False)
        results = results[results['title'] != search]    
        top_results = results.head(5)
        return(top_results)
    elif search in tv_des['title'].values:
        idx = tv_des[tv_des['title'] == search].index.item()
        for i in tv_word_binary.iloc[idx]:
            binary_list.append(i)
        point1 = np.array(binary_list).reshape(1, -1)
        point1 = [val for sublist in point1 for val in sublist]
        for j in range(len(tv)):
            binary_list2 = []
            for k in tv_word_binary.iloc[j]:
                binary_list2.append(k)
            point2 = np.array(binary_list2).reshape(1, -1)
            point2 = [val for sublist in point2 for val in sublist]
            dot_product = np.dot(point1, point2)
            norm_1 = np.linalg.norm(point1)
            norm_2 = np.linalg.norm(point2)
            cos_sim = dot_product / (norm_1 * norm_2)
            cs_list.append(cos_sim)
        tv_copy = tv_des.copy()
        tv_copy['cos_sim'] = cs_list
        results = tv_copy.sort_values('cos_sim', ascending=False)
        results = results[results['title'] != search]    
        top_results = results.head(5)
        return(top_results)
    else:
        return("Title not in dataset. Please check spelling.")

In [23]:
pd.options.display.max_colwidth = 300
recommender2('The Conjuring')

,title,description,description_filtered,cos_sim
2872,Death Note,"Light Turner finds a supernatural notebook and uses it to mete out death, attracting the attention of a detective, a demon and a girl in his class.","light turner finds supernatural notebook mete death , attracting attention detective , demon girl class .",0.460283
2193,Revenger,"Hell-bent on avenging the murder of his family, a former detective infiltrates a remote island that serves as a prison for vicious death row criminals.","hell-bent avenging murder family , former detective infiltrates remote island serves prison vicious death row criminals .",0.453147
297,A Haunted House 2,"Grieving after a tragedy, Malcolm tries to start fresh with his new girlfriend and her kids — but supernatural shenanigans find their way back to him.","grieving tragedy , malcolm start fresh girlfriend kids — supernatural shenanigans find back .",0.451781
3276,Ég man þig,"Young urbanites renovating a rundown house, and a psychiatrist grieving his son's disappearance, are connected to a supernatural, decades-old secret.","young urbanites renovating rundown house , psychiatrist grieving 's disappearance , connected supernatural , decades-old secret .",0.445435
3900,Ghost House,"While visiting the site of a supernatural shrine in the Thai countryside, a vacationing couple unwittingly upsets a vengeful demon.","while visiting site supernatural shrine thai countryside , vacationing couple unwittingly upsets vengeful demon .",0.434046


In [24]:
recommender2("Child's Play")

,title,description,description_filtered,cos_sim
3927,Good People,A struggling couple can't believe their luck when they find a stash of money in the apartment of a neighbor who was recently murdered.,a struggling couple n't believe luck find stash money apartment neighbor recently murdered .,0.431373
5152,Trip to Bhangarh: Asia's Most Haunted Place,"To amuse themselves, six college friends decide to pay a visit to a fortress believed by some to be the most haunted place in Asia.","to amuse , college friends decide pay visit fortress believed haunted place asia .",0.420084
1358,Material,"A dutiful son must hide his pursuit of stand-up comedy from his staunch father, who expects him to inherit his store and uphold their Muslim beliefs.","a dutiful hide pursuit stand-up comedy staunch father , expects inherit store uphold muslim beliefs .",0.419219
209,Rehmataan,"As unemployment, drug addiction and corruption plague a society, this drama depicts the people who believe there’s still good in the world.","as unemployment , addiction corruption plague society , drama depicts believe ’ world .",0.417483
2009,"Extremely Wicked, Shockingly Evil and Vile",Single mother Liz falls for Ted Bundy and refuses to believe the truth about his crimes for years. A drama based on a true story.,single mother liz falls ted bundy refuses believe truth crimes years . a drama based story .,0.411665


In [25]:
recommender2('Charlie and the Chocolate Factory')

,title,description,description_filtered,cos_sim
5235,Willy Wonka & the Chocolate Factory,Zany Willy Wonka causes a stir when he announces that golden tickets hidden inside his candy bars will admit holders into his secret confectionary.,zany willy wonka stir announces golden tickets hidden inside candy bars admit holders secret confectionary .,0.422200
95,Kyaa Kool Hain Hum 3,"When an unlikely porn actor falls for a woman outside the industry, he employs his co-stars as a stand-in traditional family to impress her father.","when porn actor falls woman industry , employs co-stars stand-in traditional family impress father .",0.372742
1885,Kaake Da Viyah,"In this zany comedy, a man is torn between the girl he loves and the respective women his warring mother and grandmother have chosen for him to marry.","in zany comedy , torn girl loves respective women warring mother grandmother chosen marry .",0.355643
4205,Lechmi,"The spirit of a murdered woman contacts four bachelors, enlisting their help to find her killer.","the spirit murdered woman contacts bachelors , enlisting find killer .",0.353161
3234,BoJack Horseman Christmas Special: Sabrina's Christmas Wish,"It's Christmas, and BoJack wants nothing to do with it. Then Todd shows up with a giant candy cane and an old ""Horsin' Around"" Christmas episode.","it 's christmas , bojack . then todd shows giant candy cane `` horsin ' around '' christmas episode .",0.352034


In [26]:
recommender2('Wild Child')

,title,description,description_filtered,cos_sim
4852,The Blackcoat's Daughter,"When their parents fail to pick them up for winter break, two students stay on at an all-girls boarding school in the company of a terrible presence.","when parents fail pick winter break , students stay all-girls boarding school company terrible presence .",0.446910
4545,Princess Cyd,A teenager moves to Chicago for the summer and finds a poignant mentor in her aunt and a sweet love interest in a girl who works in the neighborhood.,a teenager moves chicago summer finds poignant mentor aunt sweet love interest girl works neighborhood .,0.429377
1912,Misfit,"After living in America for years, a teenage girl moves back to the Netherlands and is quickly singled out as a misfit by the popular clique at school.","after living america years , teenage girl moves back netherlands quickly singled misfit popular clique school .",0.427247
2394,The Black Prince,"After being exiled to England as a child, a Punjabi prince struggles to return to his homeland and his Sikh faith in this historical drama.","after exiled england child , punjabi prince struggles return homeland sikh faith historical drama .",0.425307
3971,He Named Me Malala,"This documentary tells the story of a teenage Pakistani girl shot for her advocacy of women's education, her survival, and her continued efforts.","this documentary tells story teenage pakistani girl shot advocacy women 's education , survival , continued efforts .",0.421350


In [27]:
recommender2("Dr. Seuss' The Cat in the Hat")

,title,description,description_filtered,cos_sim
1431,Dark Light,"Implicated in her daughter's disappearance, a mother searches for answers when she returns to an old family home that may have an unwanted visitor.","implicated daughter 's disappearance , mother searches answers returns family home unwanted visitor .",0.500343
3972,He's Out There,"While vacationing at a remote lake house, a mother and her daughters become pawns in the twisted game of an ax-wielding psychopath.","while vacationing remote lake house , mother daughters pawns twisted game ax-wielding psychopath .",0.476212
4862,The Breadwinner,A courageous 11-year-old Afghan girl disguises herself as a boy and takes on odd jobs to provide for her family when her father is arrested.,a courageous 11-year-old afghan girl disguises boy takes odd jobs provide family father arrested .,0.471782
4227,Little Sister,"A former Goth studying to become a nun finds her faith tested when she returns home to visit her brother, who has been disfigured in the Iraq War.","a former goth studying finds faith tested returns home visit brother , disfigured iraq war .",0.471782
4060,INDIA,"A man buys a young girl, code-names her ""Doll"" and sends her to live with a mother of two who has a mysterious mission to be fulfilled.","a buys young girl , code-names `` doll '' sends live mother mysterious mission fulfilled .",0.463281


In [28]:
recommender2('Hook')

,title,description,description_filtered,cos_sim
520,Moesha,"Navigating the twists and firsts of high school, Moesha juggles friendships, romance and the occasional drama while trying to keep her family together.","navigating twists firsts high school , moesha juggles friendships , romance occasional drama family .",0.506866
1507,We're Lalaloopsy,"The Lalaloopsy gang and their adorable pets know every day is a perfect day to celebrate the magic of friendship, creativity and collaboration.","the lalaloopsy gang adorable pets day perfect day celebrate magic friendship , creativity collaboration .",0.490668
1528,Lost & Found Music Studios,"Teen singers-songwriters in an elite music program form bonds of friendship, creativity and romance while turning their passion into their profession.","teen singers-songwriters elite music program form bonds friendship , creativity romance turning passion profession .",0.482830
225,The Baker and the Beauty,"A Miami baker and a fashionista try to make romance work despite the heat they face from jealous exes, prying family members and invasive paparazzi.","a miami baker fashionista make romance work despite heat jealous exes , prying family members invasive paparazzi .",0.474750
524,Toradora!,"Menacing in appearance but gentle at heart, Ryūji forms an unlikely friendship with Taiga, a tiny girl with a sharp tongue and a tough attitude.","menacing appearance gentle heart , ryūji forms friendship taiga , tiny girl sharp tongue tough attitude .",0.462037


In [29]:
recommender2('Holidate')

,title,description,description_filtered,cos_sim
5255,Yanda Kartavya Aahe,"Thanks to an arranged marriage that was designed to make an ailing grandmother happy, newlyweds Rahul and Swati are virtually strangers. Can a four-day honeymoon make a difference when it comes to the couple's feelings about each other?","thanks arranged marriage designed make ailing grandmother happy , newlyweds rahul swati virtually strangers . can four-day honeymoon make difference couple 's feelings ?",0.471041
4114,Just Friends,"Years after high school, a once-portly nice guy comes home for the holidays and reunites with a former crush he still has feelings for.","years high school , once-portly nice guy home holidays reunites former crush feelings .",0.464953
4575,Red Christmas,A family Christmas gathering at an isolated Australian estate turns into the holiday from hell after a mysterious stranger turns up at the door.,a family christmas gathering isolated australian estate turns holiday hell mysterious stranger turns .,0.461555
2810,The Hateful Eight,"Years after the Civil War, a bounty hunter and his captive are waylaid by a Wyoming blizzard and hole up in a way station with six dicey strangers.","years civil war , bounty hunter captive waylaid wyoming blizzard hole station dicey strangers .",0.448343
2580,Us and Them,"Two strangers meet on a train and form a bond that evolves over the years. After a separation, they reconnect and reflect on their love for each other.","two strangers meet train form bond evolves years . after separation , reconnect reflect love .",0.441876


In [30]:
recommender2('After Life')

,title,description,description_filtered,cos_sim
1150,The Paper,A construction magnate takes over a struggling newspaper and attempts to wield editorial influence for power and personal gain.,a construction magnate takes struggling newspaper attempts wield editorial influence power personal .,0.512182
1377,Longmire,This contemporary crime thriller focuses on a Wyoming sheriff who's rebuilding his life and career following the death of his wife.,this contemporary crime thriller focuses wyoming sheriff 's rebuilding life career death wife .,0.460531
56,Hit & Run,A man searching for the truth behind his wife's death becomes caught up in a dangerous web of secrets and intrigue stretching from New York to Tel Aviv.,a searching truth wife 's death caught dangerous web secrets intrigue stretching new york tel aviv .,0.452509
1719,Gentlemen and Gangsters,"Now on the run, a writer relates his previous year's escapades when he got sucked into the thrilling, sordid orbit of boxer and jazz man Henry Morgan.","now run , writer relates previous year 's escapades sucked thrilling , sordid orbit boxer jazz henry morgan .",0.446211
1276,Welcome to the Family,"When an evicted single mom's estranged father dies, she and his second wife cover up his death after learning they've been written out of his will.","when evicted single mom 's estranged father , wife cover death learning 've written .",0.439024


In [31]:
recommender2('Anne with an E')

,title,description,description_filtered,cos_sim
1622,Bat Pat,"A curious and talkative bat finds spooky fun on the streets of Fogville, a town that's not as quiet as it seems, with a plucky girl and her brothers.","a curious talkative finds spooky fun streets fogville , town 's quiet , plucky girl brothers .",0.459231
1527,From Dusk Till Dawn,Bank-robbing brothers encounter vengeful lawmen and demons south of the border in this original series based on Robert Rodriguez' cult horror film.,bank-robbing brothers encounter vengeful lawmen demons south border original series based robert rodriguez ' cult horror film .,0.419219
700,Queen Sono,South African spy Queen Sono finds herself in a nefarious web of business and politics as she seeks to uncover the truth behind her mother's death.,south african spy queen sono finds nefarious web business politics seeks uncover truth mother 's death .,0.403921
1103,Eugenie Nights,"In 1940s Port Said, Kariman finds comfort and solace in the arms of an unhappily married man, who also happens to be her abusive husband's brother.","in 1940s port said , kariman finds comfort solace arms unhappily married , abusive husband 's brother .",0.403921
1518,Ice Fantasy,"The Ice Tribe prince journeys to a sacred mountain to vanquish evil but soon finds himself at war with the Fire Tribe, led by his long-lost brother.","the ice tribe prince journeys sacred mountain vanquish evil finds fire tribe , led long-lost brother .",0.398527
